#### In this notebook we develop a pipeline for hyperparameter tuning for UMAP + HDBSCAN.

We need to tune the following params:

UMAP:
- n_neighbors: [2, 0.25*len(df)]
- min_dist: [0, 0.99]
- n_components: [2, n_features]
- metric: [9 metrics for binary data]

HDBSCAN:
- min_cluster_size:
- min_samples: 
Note: If you wish to explore different min_cluster_size settings with a fixed min_samples value, especially for larger dataset sizes, you can cache the hard computation, and recompute only the relatively cheap flat cluster extraction using the memory parameter, which makes use of joblib
- cluster_selection_epsilon: ?
[- alpha]X
[- leaf clustering, not EOM]


##### Here we use the DBCV score, but could try others?


In [1]:
RANDOM_SEED = 42

In [2]:
from utilities import load_symptom_data
import hdbscan
import numpy as np
import pandas as pd
import time
import wandb

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [3]:
df = load_symptom_data('../data/cleaned_data_SYMPTOMS_9_13_23.csv')

In [4]:
n_iter = 2

In [5]:
# random_search = RandomizedSearchCV(
#     hdb,
#     param_distributions=hyper_params,
#     n_iter=n_iter,
#     scoring=clustering_score,
#     random_state=RANDOM_SEED
# )

# grid_search = GridSearchCV(
#     hdb,
#     param_grid=hyper_params,
#     scoring=clustering_score
# )

In [6]:
# start_time = time.time()
# random_search.fit(df)
# elapsed_time = time.time() - start_time
# print("%d fits took %.1f minutes" % (n_iter, elapsed_time/60))

In [7]:
# import itertools

# hyper_params = {
#     'penalty': ['l1', 'l2'],
#     'class_weight': [None, 'balanced'],
#     'max_iter': [500, 1000, 30]
# }


# a = hyper_params.values()
# combinations = list(itertools.product(*a))

##### Trying different approach to rescue grid search!

- To get GridSearchCV to fit and return the score for the full dataset, we need to use a predefined split with one copy of the data fro training and another copy for validation.
- We need to create our own scoring function with the correct signature (i.e. no need for y_true), as below.
- Need to make sure refit=False
- Need to make sure that random state is the same for each split ???

#### The following is basically working, but needs converting the hdbscan and different scoring metrics...
#### Also needs porting to scikit-optimize...Note: you need to specifiy the search space differently. 

#### And we need to add pipeline that includes a dim reduction algo.

### Necessary to downgrade numpy to <1.24 because skopt uses np.int :/

### Questions: should DVBC score use local value of 'metric' - problematic for comparing across different runs...

DCBV not working with CV.....

In [8]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA



from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [9]:
run = wandb.init(
        name='run1',
        project='test_clulster',
        config={}
    )

wandb: Currently logged in as: rusty-chris. Use `wandb login --relogin` to force relogin


In [10]:
ddf = pd.concat([df, df])

In [11]:
split = PredefinedSplit([0 if i < len(df) else 1 for i in range(len(ddf.index))])

In [12]:
test_id = np.array([0 if i < len(df) else 1 for i in ddf.index])

In [13]:
pca = PCA(random_state=42)

In [14]:
hdb = hdbscan.HDBSCAN(gen_min_span_tree=True, core_dist_n_jobs=4)

In [15]:
kmeans = KMeans(n_init='auto', random_state=42)

## Note: getting overflow in this version of DBCV when distances are small. Is there another implementation we can use?

Note: this code may not work for n_jobs!=1 because of the way we obtain the iterations number from the length of the otpimisation result.

#### Note: have to downgrade to skopt==0.8.1 and sklearn=0.24.2 for correct behaviour!

In [22]:
# Note: these scores use model.steps[1][1].labels_ instead of model.steps.labels_ because
# they are accessing the clustering model which is the second step in the pipeline.

def dbcv(data, labels, metric='euclidean'):
    
    if metric == None:
        metric = model.steps[1][1].get_params()['metric']
        
    return hdbscan.validity.validity_index(
            data, labels,
            metric=metric
        )

def dbcv_manhattan(data, labels):
    return dbcv(data, labels, metric='manhattan')

def silhouette(data, labels):
    num_labels = len(set(labels))
    if num_labels == 1:
        print("Warning: Valid number of clusters must be 2 or more.")
        return 0
    else:
        return silhouette_score(data, labels)

def calinski_harabasz(data, labels):
    return metrics.calinski_harabasz_score(data, labels)

def davies_bouldin(data, labels):
    """
    Note: 0 is best. If using for CV need to use complement.
    """
    return metrics.davies_bouldin_score(data, labels)

def cv_score(model, X, score='dbcv'):
    """
    If score == 'all' we return a dictionary of all scores, which
    can be logged to wandb on each iteration. 

    Otherwise this is intended for use as a scorer in <X>SearchCV methods.
    In that case metric should be fixed to allow comparison across different runs.
    """
    score_dict = {
        'silhouette': silhouette,
        'dbcv': dbcv,
        'calinski_harabasz': calinski_harabasz,
        'davies_bouldin': davies_bouldin
    }
    
    model.fit(X)
    labels = model.steps[1][1].labels_
    data = model.steps[0][1].transform(X)

    if score == 'all':
        return {
            score_name: score_func(data, labels) 
            for score_name, score_func in score_dict.items()
        }
    else:
        return score_dict[score](data, labels)

In [23]:
hyper_params = {
    'pca__n_components': Integer(5, 100),#, 30, 45, 60],
    'hdbscan__min_samples': Integer(1,1000),
    'hdbscan__min_cluster_size':Integer(10, 2000),  
    'hdbscan__cluster_selection_method' : Categorical(['eom', 'leaf']),
    'hdbscan__cluster_selection_epsilon' : Real(0.0, 100.0),
    'hdbscan__metric' : Categorical(['euclidean', 'manhattan'])
}

# hyper_params = {
#     'pca__n_components': [5, 15],#, 30, 45, 60],
#     'kmeans__n_clusters': Integer(2, 20)
# }

In [24]:
pipe = Pipeline(steps=[('pca', pca), ('hdbscan', hdb)])
# pipe = Pipeline(steps=[('pca', pca), ('kmeans', kmeans)])

In [32]:
tunning = BayesSearchCV(
   estimator=pipe,
   search_spaces=hyper_params,
   scoring=cv_score,
   cv=split,
   n_jobs=-1,
   refit=False,
   return_train_score=True,
   n_iter=100
)

In [33]:
def wandb_callback(result):
    iter = len(result['x_iters'])
    print('Iteration %d' %iter)
    
    current_params = result['x_iters'][-1]
    pipe.set_params(**current_params)
    all_scores = cv_score(pipe, df, score='all')
    
    log_dict = {
        'best_score': result['fun'],
        'best_params': result['x'],
        'current_params': current_params
    }
    for key in all_scores.keys():
        log_dict[key] = all_scores[key]

    run.log(log_dict)
    print(log_dict)

In [34]:
start_time = time.time()
tunning.fit(ddf.to_numpy(), callback=wandb_callback)
elapsed_time = time.time() - start_time
print(elapsed_time)

Iteration 1
{'best_score': -0.0, 'best_params': [40.785559896402724, 'leaf', 'euclidean', 1082, 371, 6], 'current_params': [40.785559896402724, 'leaf', 'euclidean', 1082, 371, 6]}
Iteration 2
{'best_score': -0.0, 'best_params': [40.785559896402724, 'leaf', 'euclidean', 1082, 371, 6], 'current_params': [24.58179919299645, 'leaf', 'euclidean', 1829, 116, 86]}
Iteration 3
{'best_score': -0.0, 'best_params': [40.785559896402724, 'leaf', 'euclidean', 1082, 371, 6], 'current_params': [31.26565225628993, 'leaf', 'euclidean', 1267, 708, 44]}
Iteration 4
{'best_score': -0.0, 'best_params': [40.785559896402724, 'leaf', 'euclidean', 1082, 371, 6], 'current_params': [79.11425635472213, 'leaf', 'manhattan', 1729, 107, 41]}
Iteration 5
{'best_score': -0.0, 'best_params': [40.785559896402724, 'leaf', 'euclidean', 1082, 371, 6], 'current_params': [38.92652977171077, 'eom', 'euclidean', 1183, 719, 21]}
Iteration 6
{'best_score': -0.0, 'best_params': [40.785559896402724, 'leaf', 'euclidean', 1082, 371, 

In [35]:
tunning.total_iterations

600

In [36]:
tunning.best_score_

0.0015346835816272564

In [37]:
tunning.best_params_

OrderedDict([('hdbscan__cluster_selection_epsilon', 11.297198441567561),
             ('hdbscan__cluster_selection_method', 'leaf'),
             ('hdbscan__metric', 'manhattan'),
             ('hdbscan__min_cluster_size', 10),
             ('hdbscan__min_samples', 1),
             ('pca__n_components', 86)])

In [38]:
tunning.cv_results_

{'mean_fit_time': array([ 0.87968397,  5.52793491,  3.0906086 ,  2.16181469,  2.36959672,
         3.05546367,  2.35229564,  3.87571192,  5.02105105,  2.94125283,
         8.94979966,  8.4944706 ,  6.31838393,  6.50117517,  7.71719825,
         9.6145246 ,  1.98101163,  9.01025188,  1.27939451,  1.84042978,
         0.52769351,  2.01433778,  1.74589515,  6.28040612,  3.06422126,
         4.12865126,  6.25315332,  6.10799062,  6.65483296,  8.37167907,
         6.9490844 ,  1.18822956,  2.38316977,  1.55593276,  3.23842287,
         7.79036915,  0.70613444,  7.46931827,  8.92806423,  2.7994622 ,
         2.6425041 ,  7.20878756,  9.20645642,  5.07250786,  2.81299865,
         4.31167495,  2.50519228,  9.41244066, 10.25694609,  0.91282749,
         1.69133806,  9.11597037,  1.72891593,  8.67573261,  6.32467747,
         3.73804021,  1.44115162,  2.28145802,  1.00080383,  7.88540518,
         2.0854857 ,  0.88465416,  7.20736814,  7.9217838 ,  9.96428859,
         1.01625335,  8.0158844 , 

In [339]:
pipe = Pipeline(steps=[('pca', pca), ('hdbscan', hdb)])
# pipe = Pipeline(steps=[('pca', pca), ('kmeans', kmeans)])
pipe.set_params(**tunning.best_params_)

Pipeline(steps=[('pca', PCA(n_components=42, random_state=42)),
                ('hdbscan',
                 HDBSCAN(cluster_selection_epsilon=4.46182243476504,
                         cluster_selection_method='leaf',
                         gen_min_span_tree=True, min_cluster_size=603,
                         min_samples=382))])

In [348]:
def cv_results_sanity_check(pipe, df, cv_results):

    bs = tunning.best_score_
    bp = tunning.best_params_
    
    pipe.set_params(**bp)

    try:
        assert bs == cv_score(pipe, df.to_numpy())
    except:
        print(bs, cv_score(pipe, df.to_numpy()))
    bid = np.where(tunning.cv_results_['mean_test_score'] == bs)[0][0]

    assert bp == tunning.cv_results_['params'][bid]
    assert bs == tunning.cv_results_['split0_test_score'][bid]
    assert bs == tunning.cv_results_['split1_test_score'][bid]
    assert bs == tunning.cv_results_['split0_train_score'][bid]
    assert bs == tunning.cv_results_['split1_train_score'][bid]

    for i, s in enumerate(tunning.cv_results_['split0_test_score']):
        assert (
            s == tunning.cv_results_['split1_test_score'][i]
        )

    print("These search results passed all sanity checks. They are deterministic and consistent. :)")

In [349]:
cv_results_sanity_check(pipe, df, tunning.cv_results_)

These search results passed all sanity checks. They are deterministic and consistent. :)


In [350]:
cv_score(pipe, df.to_numpy())

0

In [2]:
from skopt import BayesSearchCV

from sklearn.datasets import load_iris
from sklearn.svm import SVC

X, y = load_iris(return_X_y=True)

searchcv = BayesSearchCV(
    SVC(gamma='scale'),
    search_spaces={'C': (0.01, 100.0, 'log-uniform')},
    n_iter=10,
    cv=3
)

# callback handler
def on_step(optim_result):
    score = searchcv.best_score_
    print("best score: %s" % score)
    if score >= 0.98:
        print('Interrupting!')
        return True


searchcv.fit(X, y, callback=on_step)

ImportError: cannot import name 'MaskedArray' from 'sklearn.utils.fixes' (/home/expert/Documents/contracting/LCC/PLR/Chris/venv/lib/python3.10/site-packages/sklearn/utils/fixes.py)